In [0]:
import logging
import os, random, hashlib, json
import numpy as np
from datetime import datetime

# 1. Setup Logging
log_dir = "logs"
if not os.path.exists(log_dir): os.makedirs(log_dir)

log_path = f"{log_dir}/run_{datetime.now().strftime('%Y%m%d_%H%M')}.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    handlers=[logging.FileHandler(log_path), logging.StreamHandler()]
)
logging.info("Starting ETL process...")

# 2. Fix Seeds
os.environ['PYTHONHASHSEED'] = '0'
random.seed(0)
np.random.seed(0)

# 3. Hash CSVs for Reproducibility
hashes = {}
for file in ['menu_items.csv', 'order_details.csv']:
    path = f"data/{file}"
    with open(path, "rb") as f:
        hashes[file] = hashlib.sha256(f.read()).hexdigest()
    logging.info(f"Hash for {file}: {hashes[file]}")

with open("data_hashes.json", "w") as f:
    json.dump(hashes, f)

# 4. Capture Environment
%pip freeze > requirements.txt

In [0]:
import pandas as pd
import logging
from datetime import datetime

def run_etl():
    logging.info("Starting data loading process...")
    
    # 1. Load Data
    menu_df = pd.read_csv("data/menu_items.csv")
    orders_df = pd.read_csv("data/order_details.csv")

    # 2. Clean 
    menu_df['item_name'] = menu_df['item_name'].str.strip()
    menu_df['category'] = menu_df['category'].str.strip()
    
    # Handle mixed date formats (e.g., 1/1/23)
    orders_df['order_date'] = pd.to_datetime(orders_df['order_date'], format='mixed')
    
    # Robust Time Handling
    orders_df['order_time'] = orders_df['order_time'].astype(str).str.strip()
    orders_df['order_time'] = orders_df['order_time'].apply(lambda x: x if len(x) > 5 else f"{x}:00")
    orders_df['order_time'] = pd.to_timedelta(orders_df['order_time'])
    
    orders_df = orders_df.dropna()

    # 3. Join
    df = pd.merge(orders_df, menu_df, left_on='item_id', right_on='menu_item_id')

    # 4. Tidy Table 
    tidy_df = df[['order_id', 'order_date', 'order_time', 'item_name', 'category', 'price']].copy()
    
    # 5. Compute Metrics
    top_5 = tidy_df['item_name'].value_counts().nlargest(5)    
    rev_by_cat = tidy_df.groupby('category')['price'].sum()    
    tidy_df['hour'] = tidy_df['order_time'].dt.components['hours']
    busiest_hour = tidy_df['hour'].value_counts().idxmax()

    logging.info(f"Metrics computed. Busiest hour identified: {busiest_hour}:00")

    # 6. Assert Tests 
    assert not tidy_df.empty, "Error: The resulting DataFrame is empty."
    assert 'price' in tidy_df.columns, "Error: Price column is missing after join."
    
    # 7. Save Results 
    timestamp = datetime.now().strftime('%Y%m%d_%H%M')
    output_file = f"data/metrics_{timestamp}.csv"
    rev_by_cat.to_csv(output_file)
    
    logging.info(f"ETL successful. Metrics saved to: {output_file}")
    return tidy_df

final_data = run_etl()